In [ ]:
def get_ordered_signed_azimuthal(events_vector, weights, *additional_vectors):
    if(len(additional_vectors)==0):
        ordered_eta_mask_1 = (events_vector[:,0]).eta > (events_vector[:,1]).eta
        ordered_events_1 = events_vector[ordered_eta_mask_1]
        weights_1 = weights[ordered_eta_mask_1]
        signed_azimuthal_1 = ordered_events_1[:,0].phi - ordered_events_1[:,1].phi

        ordered_events_2 = events_vector[~ordered_eta_mask_1]
        weights_2 = weights[~ordered_eta_mask_1]
        signed_azimuthal_2 = ordered_events_2[:,1].phi - ordered_events_2[:,0].phi
    
    else:
        additional_events_vector_1 = additional_vectors[0]
        ordered_eta_mask_1 = (events_vector[:,0]).eta > (additional_events_vector_1[:,0]).eta
        ordered_events_1 = events_vector[ordered_eta_mask_1]
        additional_ordered_events_1 = additional_events_vector_1[ordered_eta_mask_1]
        weights_1 = weights[ordered_eta_mask_1]
        signed_azimuthal_1 = ordered_events_1[:,0].phi - additional_ordered_events_1[:,0].phi
        
        ordered_events_2 = events_vector[~ordered_eta_mask_1]
        additional_ordered_events_2 = additional_events_vector_1[~ordered_eta_mask_1]
        weights_2 = weights[~ordered_eta_mask_1]
        signed_azimuthal_2 = ordered_events_2[:,0].phi - additional_ordered_events_2[:,0].phi

    signed_azimuthal = ak.concatenate([signed_azimuthal_1, signed_azimuthal_2], mergebool=True, highlevel=True)
    signed_weights = ak.flatten(ak.concatenate([weights_1, weights_2], mergebool=True, highlevel=True))
    signed_azimuthal, signed_weights = signed_azimuthal_overflow(signed_azimuthal, signed_weights)

    return signed_azimuthal, signed_weights

def signed_azimuthal_overflow(signed_azimuthal, signed_weights):
    if np.any((signed_azimuthal < -2 * np.pi) | (signed_azimuthal > 2 * np.pi)):
        raise ValueError("Azimuthal angle(s) exceed the range [-2π, 2π].")
            
    plus_overflow_mask = signed_azimuthal > np.pi
    plus_overflow = signed_azimuthal[plus_overflow_mask]
    plus_overflow_weights = signed_weights[plus_overflow_mask]

    minus_overflow_mask = signed_azimuthal < -np.pi
    minus_overflow = signed_azimuthal[minus_overflow_mask]
    minus_overflow_weights = signed_weights[minus_overflow_mask]

    overflow_mask = plus_overflow_mask | minus_overflow_mask

    not_overflowed = signed_azimuthal[~overflow_mask]
    not_overflow_weights = signed_weights[~overflow_mask]

    shifted_plus_overflow = plus_overflow - (2 * np.pi)
    shifted_minus_overflow = minus_overflow + (2 * np.pi)

    signed_azimuthal = ak.concatenate([not_overflowed, shifted_plus_overflow, shifted_minus_overflow], mergebool=True, highlevel=True)
    signed_weights = ak.concatenate([not_overflow_weights, plus_overflow_weights, minus_overflow_weights], mergebool=True, highlevel=True)

    return signed_azimuthal, signed_weights